In [1]:
import numpy as np
import diffusionstuff7 as ds

C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


In [2]:
#Setting up the 2D system

niter = 1
center_reduction = 1.0
sigmastepmax = 0.20

nx = 500 # Number of points in simulation box
xmax = 50 # range of x
xs = np.linspace(0, xmax, nx)

ny = nx
ymax = xmax
ys = np.linspace(0, ymax, ny)

deltaX = xs[1]-xs[0]
deltaY = ys[1]-ys[0]

Nbar = 1.0 # new Nbar from VMD, 260K
Nstar = .9/(2*np.pi)
# Initialize as a pre-equilibrated layer of liquid over ice

Nice = np.ones((nx,ny))


Fliq = ds.getNliq_2d_array(Nice,Nstar,Nbar,niter)

#simgastep_2d = ds.getsigmastep_2d(xs,xmax,center_reduction,sigmastepmax)

# Lay out the system
Ntot = Fliq + Nice
# nmid = int(nx/2)
# nquart = int(nx/4)
# xmid = max(x)/2
# xmax = x[nx-1]
Nice_start = Nice[0]

In [15]:
from numba import njit,float64,int32
import numba as nb

@njit(nb.types.containers.UniTuple(float64[:,:],2)(float64[:],float64[:]))
def meshgrid(x, y):
    xx = np.empty(shape=(x.size, y.size), dtype=x.dtype)
    yy = np.empty(shape=(x.size, y.size), dtype=y.dtype)
    for i in range(y.size):
        for j in range(x.size):
            xx[i,j] = x[j]  # change to x[k] if indexing xy
            yy[i,j] = y[i]  # change to y[j] if indexing xy
    return xx, yy

In [18]:
#testing sigmastep 2d

xs = np.linspace(0, xmax, nx)
ys = np.linspace(0, ymax, ny)

sigmapfac = 1-center_reduction/100 #float64
xmid = max(xs)/2 #float64
ymid = max(ys)/2 #float64
x_alt = xs
y_alt = ys
(xs,ys) = meshgrid(xs,ys)



fsig = ((xs-xmid)**2 + (ys-ymid)**2)/xmid**2*(1-sigmapfac)+sigmapfac #NOTE xmid in denominator does not support distinct 2d discretization (diff dx and dy)

print(xs,ys)

#print(type(meshgrid(xs,ys)))

xs = np.linspace(0, xmax, nx)
ys = np.linspace(0, ymax, ny)
my_xs,my_ys = meshgrid(xs,ys)
print(my_xs,my_ys)


[[ 0.         0.1002004  0.2004008 ... 49.7995992 49.8997996 50.       ]
 [ 0.         0.1002004  0.2004008 ... 49.7995992 49.8997996 50.       ]
 [ 0.         0.1002004  0.2004008 ... 49.7995992 49.8997996 50.       ]
 ...
 [ 0.         0.1002004  0.2004008 ... 49.7995992 49.8997996 50.       ]
 [ 0.         0.1002004  0.2004008 ... 49.7995992 49.8997996 50.       ]
 [ 0.         0.1002004  0.2004008 ... 49.7995992 49.8997996 50.       ]] [[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.1002004  0.1002004  0.1002004 ...  0.1002004  0.1002004  0.1002004]
 [ 0.2004008  0.2004008  0.2004008 ...  0.2004008  0.2004008  0.2004008]
 ...
 [49.7995992 49.7995992 49.7995992 ... 49.7995992 49.7995992 49.7995992]
 [49.8997996 49.8997996 49.8997996 ... 49.8997996 49.8997996 49.8997996]
 [50.        50.        50.        ... 50.        50.        50.       ]]
[[ 0.         0.1002004  0.2004008 ... 49.7995992 49.8997996 50.       ]
 [ 0.         0.1002004  0.2004008 ... 